# Module 2.3: Momentum and Oscillators

**Topics:** RSI, Stochastic, Williams %R, CCI
**Key concepts:** Overbought/oversold conditions, divergences

This notebook delves into momentum indicators and oscillators, which are crucial for assessing the velocity of price movements and identifying potential trend reversals.

## 1. Introduction to Momentum and Oscillators

Momentum indicators measure the rate of change in security prices. Oscillators are a type of momentum indicator that fluctuate above and below a centerline or between set levels as their values change over time. They are often used to identify overbought or oversold conditions, spot divergences, and generate buy/sell signals.

In [13]:
# Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
import requests
import os
from plotly.subplots import make_subplots
from datetime import datetime


## 2. Fetching Data

Let's fetch some historical stock data to work with. We'll use `yfinance` to get data for a sample stock.

In [27]:
symbol = 'AAPL'
start_date = '2025-01-01'
end_date = datetime.now().strftime("%Y-%m-%d")
session = requests.Session()
proxy_host = "localhost"
proxy_port = 10809
proxy_url = f"http://{proxy_host}:{proxy_port}"
session.proxies.update(
    {
        "http": proxy_url,
        "https": proxy_url,
    }
)
os.environ['HTTP_PROXY'] = proxy_url
os.environ['HTTPS_PROXY'] = proxy_url
ticker = yf.Ticker(symbol)
ticker.session = session
# Download data
data = ticker.history(start=start_date, end=end_date, interval='1d')
data = data.round(2)

print(f"Downloaded {len(data)} days of data for {symbol}")
print(f"Date range: {data.index[0].date()} to {data.index[-1].date()}")
print("\nFirst 5 rows:")
data.head()

Downloaded 132 days of data for AAPL
Date range: 2025-01-02 to 2025-07-15

First 5 rows:


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-01-02 00:00:00-05:00,248.33,248.50,241.24,243.26,55740700,0.0,0.0
2025-01-03 00:00:00-05:00,242.77,243.59,241.31,242.77,40244100,0.0,0.0
2025-01-06 00:00:00-05:00,243.72,246.73,242.61,244.41,45045600,0.0,0.0
2025-01-07 00:00:00-05:00,242.40,244.96,240.77,241.63,40856000,0.0,0.0
2025-01-08 00:00:00-05:00,241.34,243.12,239.47,242.12,37628900,0.0,0.0


## 3. Relative Strength Index (RSI)

The Relative Strength Index (RSI) is a momentum oscillator that measures the speed and change of price movements. RSI oscillates between zero and 100. Traditionally, RSI is considered overbought when above 70 and oversold when below 30.

### Formula
The RSI is calculated using the following formula:
$$
RSI = 100 - \frac{100}{1 + RS}
$$
Where:
- $RS = \frac{\text{Average Gain}}{\text{Average Loss}}$
- **Average Gain**: The average of the upward price changes over a specified period.
- **Average Loss**: The average of the downward price changes over the same period.
The averages are typically smoothed, often using an exponential moving average.

In [28]:
# Calculate RSI
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

data['RSI'] = calculate_rsi(data)

## 4. Stochastic Oscillator

The Stochastic Oscillator is a momentum indicator that compares a particular closing price of a security to a range of its prices over a certain period of time. The sensitivity of the oscillator to market movements is reducible by adjusting that time period or by taking a moving average of the result.

### Formula
The Stochastic Oscillator is calculated with these formulas:
$$
\%K = 100 \times \frac{C - L_{14}}{H_{14} - L_{14}}
$$
$$
\%D = \text{3-period SMA of } \%K
$$
Where:
- $C$: The most recent closing price.
- $L_{14}$: The lowest price over the last 14 periods.
- $H_{14}$: The highest price over the last 14 periods.
- $\%D$: The 3-day simple moving average of %K, which acts as a signal line.

In [29]:
# Calculate Stochastic Oscillator
def calculate_stochastic_oscillator(data, window=14):
    low_min = data['Low'].rolling(window=window).min()
    high_max = data['High'].rolling(window=window).max()
    
    data['%K'] = 100 * ((data['Close'] - low_min) / (high_max - low_min))
    data['%D'] = data['%K'].rolling(window=3).mean()
    return data

data = calculate_stochastic_oscillator(data)

## 5. Williams %R

Williams %R is a momentum indicator that is the inverse of the Fast Stochastic Oscillator. Also referred to as %R, Williams %R reflects the level of the close relative to the highest high for the look-back period.

### Formula
The Williams %R is calculated as follows:
$$
\%R = -100 \times \frac{H_{14} - C}{H_{14} - L_{14}}
$$
Where:
- $C$: The most recent closing price.
- $L_{14}$: The lowest price over the last 14 periods.
- $H_{14}$: The highest price over the last 14 periods.

In [30]:
# Calculate Williams %R
def calculate_williams_r(data, window=14):
    high_max = data['High'].rolling(window=window).max()
    low_min = data['Low'].rolling(window=window).min()

    data['%R'] = -100 * ((high_max - data['Close']) / (high_max - low_min))
    return data

data = calculate_williams_r(data)

## 6. Commodity Channel Index (CCI)

The Commodity Channel Index (CCI) is a versatile indicator that can be used to identify a new trend or warn of extreme conditions. CCI measures the current price level relative to an average price level over a given period of time.

### Formula
The CCI is calculated with the following steps:
1. Calculate the Typical Price (TP): 
   $$
   TP = \frac{\text{High} + \text{Low} + \text{Close}}{3}
   $$
2. Calculate the Simple Moving Average (SMA) of the TP.
3. Calculate the Mean Deviation.
4. Compute the CCI:
   $$
   CCI = \frac{TP - \text{SMA}(TP)}{0.015 \times \text{Mean Deviation}}
   $$

In [31]:
# Calculate CCI
def calculate_cci(data, window=20):
    tp = (data['High'] + data['Low'] + data['Close']) / 3
    sma_tp = tp.rolling(window=window).mean()
    mad = (tp - sma_tp).abs().rolling(window=window).mean()
    
    data['CCI'] = (tp - sma_tp) / (0.015 * mad)
    return data

data = calculate_cci(data)

## 7. Visualization and Interpretation

Now let's plot these indicators and see how they can be interpreted. We will look for overbought/oversold signals and potential divergences.

- **RSI**: Overbought > 70, Oversold < 30
- **Stochastic**: Overbought > 80, Oversold < 20
- **Williams %R**: Overbought > -20, Oversold < -80
- **CCI**: Overbought > 100, Oversold < -100

In [33]:
# Create subplots
fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                    subplot_titles=(f'{ticker} Price', 'RSI', 'Stochastic Oscillator (%K and %D)', 'Williams %R', 'CCI'),
                    row_heights=[0.4, 0.15, 0.15, 0.15, 0.15])

# Price chart
fig.add_trace(go.Candlestick(x=data.index, open=data['Open'], high=data['High'],
                             low=data['Low'], close=data['Close'], name='Price'), row=1, col=1)

# RSI
fig.add_trace(go.Scatter(x=data.index, y=data['RSI'], name='RSI'), row=2, col=1)
fig.add_hline(y=70, row=2, col=1, line_dash="dash", line_color="red")
fig.add_hline(y=30, row=2, col=1, line_dash="dash", line_color="green")

# Stochastic Oscillator
fig.add_trace(go.Scatter(x=data.index, y=data['%K'], name='%K (Fast)'), row=3, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['%D'], name='%D (Slow)'), row=3, col=1)
fig.add_hline(y=80, row=3, col=1, line_dash="dash", line_color="red")
fig.add_hline(y=20, row=3, col=1, line_dash="dash", line_color="green")

# Williams %R
fig.add_trace(go.Scatter(x=data.index, y=data['%R'], name='%R'), row=4, col=1)
fig.add_hline(y=-20, row=4, col=1, line_dash="dash", line_color="red")
fig.add_hline(y=-80, row=4, col=1, line_dash="dash", line_color="green")

# CCI
fig.add_trace(go.Scatter(x=data.index, y=data['CCI'], name='CCI'), row=5, col=1)
fig.add_hline(y=100, row=5, col=1, line_dash="dash", line_color="red")
fig.add_hline(y=-100, row=5, col=1, line_dash="dash", line_color="green")


fig.update_layout(height=1200, title_text="Momentum Oscillators Analysis", legend_title_text='Lines')
fig.update_xaxes(rangeslider_visible=False)
fig.show()

## 8. Divergences

A divergence occurs when the price of an asset is moving in the opposite direction of a technical indicator. It's a warning sign that the current price trend may be weakening, and in some cases may lead to the price changing direction.

- **Bullish Divergence**: Price records a lower low, while the indicator shows a higher low. This can signal a potential upturn.
- **Bearish Divergence**: Price records a higher high, while the indicator makes a lower high. This can signal a potential downturn.

Identifying divergences programmatically can be complex and is a good topic for a more advanced notebook. For now, try to spot them visually on the chart above.

## Conclusion

In this notebook, we implemented and visualized four popular momentum oscillators. These tools are powerful for identifying overbought/oversold conditions and potential trend reversals through divergences. Remember that no single indicator is perfect, and they are best used in combination with other analysis techniques.